In [17]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

np.random.seed(1)
torch.manual_seed(1)

In [11]:
train = pd.read_csv('cleantrain.csv')
test = pd.read_csv('cleantest.csv')

/tmp/ipykernel_12378/2146946413.py:1: DtypeWarning: Columns (17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('cleantrain.csv')
/tmp/ipykernel_12378/2146946413.py:2: DtypeWarning: Columns (16,17,18,19,20,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('cleantest.csv')


In [3]:
train

,Target,country_code,demog_1,demog_2,income,demog_3,city_tier,occupation,demog_4,demog_5,...,demog_39,email_domain,demog_40,demog_41,others_42,others_43,others_44,others_45,demog_42,demog_43
0,1,IN,55.0,1.0,100001 to 5L,101.0,Tier 2,Other,N,4.0,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
1,1,IN,55.0,2.0,100001 to 5L,101.0,Tier 8,Self_Employed,N,NaN,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
2,1,IN,53.0,2.0,5L to 10L,101.0,Tier 1,Self_Employed,N,4.0,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,medium
3,1,IN,55.0,3.0,10L to 25L,101.0,Tier 1,Self_Employed,N,3.0,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,High
4,1,IN,56.0,2.0,100001 to 5L,101.0,Tier 7,Self_Employed,N,2.0,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66571,0,IN,55.0,1.0,100001 to 5L,101.0,Tier 3,Other,N,3.0,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,medium
66572,0,IN,401.0,4.0,0 to 1L,103.0,Tier 4,Homemaker,N,3.0,...,1,Others,High,1,NaN,NaN,NaN,NaN,1,medium
66573,0,IN,55.0,6.0,0 to 1L,101.0,Tier 7,Student,N,4.0,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium
66574,0,IN,56.0,2.0,0 to 1L,101.0,Tier 5,Self_Employed,N,2.0,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium


In [6]:
txn_cols = train.filter(regex='txn').columns
txn_cols

Index(['txn_1', 'txn_2', 'txn_3', 'txn_4', 'txn_5', 'txn_6', 'txn_7', 'txn_8',
       'txn_9', 'txn_10', 'txn_11', 'txn_12', 'txn_13', 'txn_14', 'txn_15',
       'txn_16', 'txn_17', 'txn_18', 'txn_19', 'txn_20', 'txn_21', 'txn_22',
       'txn_23', 'txn_24', 'txn_25', 'txn_26', 'txn_27', 'txn_28', 'txn_29',
       'txn_30', 'txn_31', 'txn_32', 'txn_33', 'txn_34', 'txn_35', 'txn_36',
       'txn_37', 'txn_38', 'txn_39', 'txn_40', 'txn_41', 'txn_42', 'txn_43',
       'txn_44', 'txn_45', 'txn_46', 'txn_47', 'txn_48', 'txn_49', 'txn_50',
       'txn_51', 'txn_52', 'txn_53', 'txn_54', 'txn_55', 'txn_56', 'txn_57',
       'txn_58', 'txn_59', 'txn_60', 'txn_61', 'txn_62', 'txn_63', 'txn_64',
       'txn_65', 'txn_66', 'txn_67', 'txn_68', 'txn_69', 'txn_70', 'txn_71',
       'txn_72', 'txn_73', 'txn_74', 'txn_75', 'txn_76', 'txn_77', 'txn_78',
       'txn_79', 'txn_80', 'txn_81'],
      dtype='object')

In [12]:
y_train = train['Target']
train.drop(columns=['Target','country_code'], inplace=True)
X_train1 = train.drop(columns=txn_cols)
X_train2 = train[txn_cols]

y_test = test['Target']
test.drop(columns=['Target','country_code'], inplace=True)
X_test1 = test.drop(columns=txn_cols)
X_test2 = test[txn_cols]

In [13]:
X_train1

,demog_1,demog_2,income,demog_3,city_tier,occupation,demog_4,demog_5,demog_6,demog_7,...,demog_39,email_domain,demog_40,demog_41,others_42,others_43,others_44,others_45,demog_42,demog_43
0,55.0,1.0,100001 to 5L,101.0,Tier 2,Other,N,4.0,1.0,1.0,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
1,55.0,2.0,100001 to 5L,101.0,Tier 8,Self_Employed,N,NaN,1.0,1.0,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
2,53.0,2.0,5L to 10L,101.0,Tier 1,Self_Employed,N,4.0,1.0,1.0,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,medium
3,55.0,3.0,10L to 25L,101.0,Tier 1,Self_Employed,N,3.0,1.0,1.0,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,High
4,56.0,2.0,100001 to 5L,101.0,Tier 7,Self_Employed,N,2.0,1.0,1.0,...,1,gmail,High,1,NaN,NaN,NaN,NaN,1,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66571,55.0,1.0,100001 to 5L,101.0,Tier 3,Other,N,3.0,1.0,1.0,...,1,gmail,medium,1,NaN,NaN,NaN,NaN,1,medium
66572,401.0,4.0,0 to 1L,103.0,Tier 4,Homemaker,N,3.0,1.0,1.0,...,1,Others,High,1,NaN,NaN,NaN,NaN,1,medium
66573,55.0,6.0,0 to 1L,101.0,Tier 7,Student,N,4.0,1.0,1.0,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium
66574,56.0,2.0,0 to 1L,101.0,Tier 5,Self_Employed,N,2.0,1.0,1.0,...,1,gmail,low,1,NaN,NaN,NaN,NaN,1,medium


In [14]:
X_train2

,txn_1,txn_2,txn_3,txn_4,txn_5,txn_6,txn_7,txn_8,txn_9,txn_10,...,txn_72,txn_73,txn_74,txn_75,txn_76,txn_77,txn_78,txn_79,txn_80,txn_81
0,10.0,10.0,0.0,0.0,35.0,35.0,55.0,55.0,4.0,4.0,...,0.0,69.0,69.0,1658.0,1658.0,0.0,0.0,0.0,1621.0,32.0
1,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,...,0.0,14.0,14.0,8.0,8.0,1.0,0.0,0.0,8.0,8.0
2,0.0,0.0,1.0,1.0,0.0,0.0,5.0,5.0,24.0,24.0,...,0.0,27.0,27.0,18.0,18.0,0.0,1.0,1.0,15.0,22.0
3,10.0,12.0,3.0,4.0,10.0,61.0,10.0,18.0,249.0,252.0,...,0.0,297.0,353.0,48.0,66.0,0.0,0.0,0.0,62.0,184.0
4,0.0,0.0,0.0,0.0,95.0,95.0,1.0,1.0,0.0,0.0,...,0.0,95.0,95.0,3689.0,3689.0,0.0,0.0,0.0,3634.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,16.0,...,0.0,8.0,29.0,2.0,10.0,13.0,4.0,4.0,3.0,5.0
66572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,3.0,22.0,4.0,4.0,NaN,NaN
66573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
66574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Taking the last time step output
        return out


class LinearModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearModel, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.fc(x)
        return out


class CombinedModel(nn.Module):
    def __init__(self, rnn_model, linear_model, input_size):
        super(CombinedModel, self).__init__()
        self.rnn_model = rnn_model
        self.linear_model = linear_model
        self.final_linear = nn.Linear(input_size, 1)

    def forward(self, x_rnn, x_linear):
        out_rnn = self.rnn_model(x_rnn)
        out_linear = self.linear_model(x_linear)
        combined_output = torch.cat((out_rnn, out_linear), dim=1)
        final_output = self.final_linear(combined_output)
        return final_output


# Instantiate the models
rnn_model = RNNModel(input_size=1, hidden_size=4, output_size=2)
linear_model = LinearModel(input_size=93, output_size=8)
combined_model = CombinedModel(rnn_model, linear_model, 10)
